# The construction of Relational Cazalla Final Dataset

## 1. Extraction of the phrases. Subjet to changes with Diego, David and Josiane

In [1]:
import lxml.etree as ET
import regex as re
import pandas as pd

In [2]:
tree = ET.parse(r'/home/tensorboard/Documentos/D4R/7. Cazalla datasets/ProcesoCazalla-NoComents (1).xml')
root = tree.getroot()
parragraph_objetives = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}p']
entity_objetives = [
        "{http://www.tei-c.org/ns/1.0}persName",
        "{http://www.tei-c.org/ns/1.0}date",
        "{http://www.tei-c.org/ns/1.0}orgName",
        "{http://www.tei-c.org/ns/1.0}placeName",
       "{http://www.tei-c.org/ns/1.0}rs"]


entity_objetives_dict ={
        "{http://www.tei-c.org/ns/1.0}persName": '[PERSON]',
        "{http://www.tei-c.org/ns/1.0}date": '[DATE]',
        "{http://www.tei-c.org/ns/1.0}orgName": '[ORG]',
        "{http://www.tei-c.org/ns/1.0}placeName": '[PLACE]',
       "{http://www.tei-c.org/ns/1.0}rs": '[PERSON_REFERENCE]'
       }

specials = ["{http://www.tei-c.org/ns/1.0}fw", "{http://www.tei-c.org/ns/1.0}pb"]

In [ ]:
type(list(entity_objetives_dict.keys()))    

## Extracción para los historiadores

In [ ]:
result = ''
nsmap = {'tei': list(tree.getroot().nsmap.values())[0]}
for div in root.iter():
    id_ =  div.get("{http://www.w3.org/XML/1998/namespace}id", "NULL")
    xml_id = div.get("{http://www.w3.org/XML/1998/namespace}id", "")

    date = div.xpath('.//tei:p[1]/tei:date/@when', namespaces=nsmap)

    if xml_id.startswith("Vall"):
        result +=  xml_id + ' /// '
        result += str(date)+ ' /// '
        for element in div:

            if element.tag in parragraph_objetives and element.text is not None:
                # Include all text content within the <p> element
                paragraph_text = element.text

                for entity in element:
                    if entity.tag in entity_objetives and entity.text is not None:
                        for k, v in entity_objetives_dict.items():
                            if entity.tag ==k:
                        # Include entity tags as well
                                paragraph_text += v + re.sub(r'\s+', ' ', entity.text) + v if entity.tag != "{http://www.tei-c.org/ns/1.0}date" else entity.text
                    if entity.tail is not None:
                        # Include tail text of entities
                        paragraph_text += entity.tail
                paragraph_text += (element.tail or '')

                result += paragraph_text

In [ ]:
with open('texto_para_los_historiadores.txt', 'w', encoding='utf-8') as f:
    f.writelines(result)
    f.close


with open('texto_para_los_historiadores_2.txt', 'w', encoding='utf-8') as f:
    x = f.write(result)
    f.close

## extracción para el dataset

In [3]:
result = ''
nsmap = {'tei': list(tree.getroot().nsmap.values())[0]}
for div in root[1].iter():
    id_ =  div.get("{http://www.w3.org/XML/1998/namespace}id", "NULL")
    xml_id = div.get("{http://www.w3.org/XML/1998/namespace}id", "")

    date = div.xpath('.//tei:p[1]/tei:date/@when', namespaces=nsmap)

    if xml_id.startswith("Vall"):
        # Extracting the value of the 'type' attribute
        type_attribute = div.attrib.get('type', 'No type attribute found')

        # Concatenating the strings with the type attribute value
        result += 'CUT HERE '+ xml_id + ' ??? ' + type_attribute + ' /// ' + str(date)+ ' === '
        
        for element in div:

            if element.tag in parragraph_objetives and element.text is not None:
                # Include all text content within the <p> element
                paragraph_text = element.text

                for entity in element:
                    if entity.tag in entity_objetives and entity.text is not None:
                        for k, v in entity_objetives_dict.items():
                            if entity.tag ==k:
                                # Include entity tags as well
                                paragraph_text += v + re.sub(r'\s+', ' ', entity.text) + v if entity.tag != "{http://www.tei-c.org/ns/1.0}date" else entity.text
                    if entity.tail is not None:
                        # Include tail text of entities
                        paragraph_text += entity.tail
                paragraph_text += (element.tail or '')

                result += paragraph_text


In [4]:
result= result.rstrip().lstrip()
result = re.sub(r'\s+', ' ', result)
result = re.sub(r'T.º', 'Testigo ', result)
result = re.sub(r'Test.', 'Testigo', result)
result = re.sub('Testigogo', 'Testigo', result)
result = re.sub(r'(\d)\.(\s*[A-Za-z])', r'\1\2' ,result)
result= re.sub(r'2.ª', 'segunda', result)
result = re.sub(r'Dr.', 'Dr', result)


result = re.sub(r'\.', ' . ', result)
result = re.sub(r';', ' ; ', result)
result = re.sub(r',', ' , ', result)
result = re.sub(r':', ' : ', result)
result = re.sub(r'\[No declarado en el proceso\]', '', result)
result = re.sub(r'\[capítulo 4 de la acusación\]', '', result)
X=  result.split('CUT HERE' )




In [5]:
result

"CUT HERE Vall-PC-FI002 ??? Clamosa /// [] === Muy magníficos y muy poderosos Señores .  El [PERSON]Licenciado Ramírez[PERSON] ,  fiscal de este [ORG]Santo Oficio[ORG] ,  ante vuestras mercedes parezco y digo :  que por los libros y registros de este [ORG]Santo Oficio[ORG] ,  de los cuales hago presentación ,  consta y parece que [PERSON]Pedro de Cazalla[PERSON] ,  clérigo ,  cura del lugar de [PLACE]Pedrosa[PLACE] ,  hermano del [PERSON]Dr Cazalla[PERSON] ,  está notado y testificado del crimen y delito de herejía ,  del cual protesto acusarles en forma ;  a vuestras mercedes pido y suplico manden dar y den su mandamiento para que el dicho [PERSON]Pedro de Cazalla[PERSON] sea traído preso a las cárceles de este [ORG]Santo Oficio[ORG] ,  con secuestro de bienes en forma ,  sobre lo cual y en lo necesario imploro el [ORG]Santo Oficio[ORG] ,  y pido justicia .  Rúbrica CUT HERE Vall-PC-FI003 ??? anotaciones /// ['1558-04-23'] === PRESION . ratura Se dio mandamiento de prisión a 23 de abr

In [6]:
X = [ i for i in X if i != '']
X = [i.split('???') for i in X]
X = [a.split('///') for i in X for a in i]
X = [a.split('===') for i in X for a in i]
y= [a for i in X for a in i]
triplets = [y[i:i + 4] for i in range(0, len(y), 4)]

ds = {}

for i in triplets:
    ds[i[0]] = {i[1]: {i[2]: i[3].strip()}}

In [7]:
af = ds  
result_dict = {}
for identifier, attrib_dict in af.items():
    result_dict[identifier] = {}
    for attrib, date_dict in attrib_dict.items():
        
        result_dict[identifier][attrib] = {}
        #result_dict[identifier][date][value]= {}
        for date, value in date_dict.items():
            value_list = value.split('.')
            result_dict[identifier][attrib][date]={}
            result_dict[identifier][attrib][date][value]={}
        for phrase in value_list:
            entity_count = len(re.findall(r'\[(.*?)\](.*?)\[(.*?)\]', phrase))

            if entity_count not in result_dict[identifier][attrib][date][value]:
                result_dict[identifier][attrib][date][value][entity_count] = []

                result_dict[identifier][attrib][date][value][entity_count].append(phrase)
            else:
                result_dict[identifier][attrib][date][value][entity_count].append(phrase)

## 2. Creating the dataset

In [8]:
data = []

for identifier, attrib_dict in result_dict.items():
    for attrib, date_dict in attrib_dict.items():
        for date, value_list_dict in date_dict.items():

            for parragraph, entity_counter_dict in value_list_dict.items():
                for counter, phrases in entity_counter_dict.items():
                    for phrase in phrases:
                        data.append({
                        'ID': identifier,
                        'Attrib':attrib,
                        'date': date,
                        'Parragraph': parragraph,
                        'Sentence': phrase,
                        'number of entities': counter,
                        
                    })

df = pd.DataFrame(data)

In [9]:
df

,ID,Attrib,date,Parragraph,Sentence,number of entities
0,Vall-PC-FI002,Clamosa,[],Muy magníficos y muy poderosos Señores . El [...,Muy magníficos y muy poderosos Señores,0
1,Vall-PC-FI002,Clamosa,[],Muy magníficos y muy poderosos Señores . El [...,Rúbrica,0
2,Vall-PC-FI002,Clamosa,[],Muy magníficos y muy poderosos Señores . El [...,"El [PERSON]Licenciado Ramírez[PERSON] , fis...",9
3,Vall-PC-FI003,anotaciones,['1558-04-23'],PRESION . ratura Se dio mandamiento de prisión...,PRESION,0
4,Vall-PC-FI003,anotaciones,['1558-04-23'],PRESION . ratura Se dio mandamiento de prisión...,ratura Se dio mandamiento de prisión a 23 de ...,0
...,...,...,...,...,...,...
1027,Vall-PC-FC229,Fallo,['1559-10-08'],El Licenciado [PERSON]Francisco Vaca[PERSON] ....,(Rúbrica),0
1028,Vall-PC-FC229,Fallo,['1559-10-08'],El Licenciado [PERSON]Francisco Vaca[PERSON] ....,(Rúbrica),0
1029,Vall-PC-FC229,Fallo,['1559-10-08'],El Licenciado [PERSON]Francisco Vaca[PERSON] ....,,0
1030,Vall-PC-FC229,Fallo,['1559-10-08'],El Licenciado [PERSON]Francisco Vaca[PERSON] ....,Dada y pronunciada fue la dicha sentencia po...,10


In [10]:
df['date'] = df['date'].apply(lambda x: re.sub(r"\['", '', x))
df['date'] = df['date'].apply(lambda x: re.sub(r"\']", '', x))
df['date'] = df['date'].apply(lambda x: re.sub(r"\[]", '', x))
df['date'] = df['date'].apply(lambda x: re.sub(r"'\w+-\d+-\d+'", '', x))
df['date'] = df['date'].apply(lambda x: re.sub(r"'\w+-\d+-\d+", '', x))
df['date'] = df['date'].apply(lambda x: re.sub(r"'", '', x))
df['date'] = df['date'].apply(lambda x: re.sub(r",", '', x))
df['date'] = df['date'].apply(lambda x: re.sub(r" ",'', x))

In [11]:
df

,ID,Attrib,date,Parragraph,Sentence,number of entities
0,Vall-PC-FI002,Clamosa,,Muy magníficos y muy poderosos Señores . El [...,Muy magníficos y muy poderosos Señores,0
1,Vall-PC-FI002,Clamosa,,Muy magníficos y muy poderosos Señores . El [...,Rúbrica,0
2,Vall-PC-FI002,Clamosa,,Muy magníficos y muy poderosos Señores . El [...,"El [PERSON]Licenciado Ramírez[PERSON] , fis...",9
3,Vall-PC-FI003,anotaciones,1558-04-23,PRESION . ratura Se dio mandamiento de prisión...,PRESION,0
4,Vall-PC-FI003,anotaciones,1558-04-23,PRESION . ratura Se dio mandamiento de prisión...,ratura Se dio mandamiento de prisión a 23 de ...,0
...,...,...,...,...,...,...
1027,Vall-PC-FC229,Fallo,1559-10-08,El Licenciado [PERSON]Francisco Vaca[PERSON] ....,(Rúbrica),0
1028,Vall-PC-FC229,Fallo,1559-10-08,El Licenciado [PERSON]Francisco Vaca[PERSON] ....,(Rúbrica),0
1029,Vall-PC-FC229,Fallo,1559-10-08,El Licenciado [PERSON]Francisco Vaca[PERSON] ....,,0
1030,Vall-PC-FC229,Fallo,1559-10-08,El Licenciado [PERSON]Francisco Vaca[PERSON] ....,Dada y pronunciada fue la dicha sentencia po...,10


In [ ]:
from datetime import datetime
def dating(date):
    if date:
        try:
            obj = datetime.strptime(date, '%Y-%m-%d')
            return obj.date()
        except ValueError:
            # Handle the case where the date format is incorrect
            return None
    else:
        return None
df['date'] = df['date'].apply(lambda x: dating(x))

In [12]:
df['Sentence'][87]

''

In [14]:
type(df['date'][1027])

str

In [15]:
df.to_excel('complete_relations_diego_changes_13-02-2024.xlsx')

In [ ]:
import json

with open("relations_located.json", "w") as outfile: 
    json.dump(result_dict, outfile)

In [17]:
df = pd.read_excel(r'/home/tensorboard/Documentos/D4R/7. Cazalla datasets/relartions dataset/complete_relations_diego_changes_13-02-2024.xlsx')
df_1 = df
df.dropna(subset=['Sentence'], inplace=True)

# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)
df_1['Phrase_tokenized'] = [sentence.split() for sentence in df_1['Sentence']]


In [18]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          889 non-null    int64 
 1   ID                  889 non-null    object
 2   Attrib              889 non-null    object
 3   date                881 non-null    object
 4   Parragraph          889 non-null    object
 5   Sentence            889 non-null    object
 6   number of entities  889 non-null    int64 
 7   Phrase_tokenized    889 non-null    object
dtypes: int64(2), object(6)
memory usage: 55.7+ KB


In [19]:
df_1

,Unnamed: 0,ID,Attrib,date,Parragraph,Sentence,number of entities,Phrase_tokenized
0,0,Vall-PC-FI002,Clamosa,NaN,Muy magníficos y muy poderosos Señores . El [...,Muy magníficos y muy poderosos Señores,0,"[Muy, magníficos, y, muy, poderosos, Señores]"
1,1,Vall-PC-FI002,Clamosa,NaN,Muy magníficos y muy poderosos Señores . El [...,Rúbrica,0,[Rúbrica]
2,2,Vall-PC-FI002,Clamosa,NaN,Muy magníficos y muy poderosos Señores . El [...,"El [PERSON]Licenciado Ramírez[PERSON] , fis...",9,"[El, [PERSON]Licenciado, Ramírez[PERSON], ,, f..."
3,3,Vall-PC-FI003,anotaciones,1558-04-23,PRESION . ratura Se dio mandamiento de prisión...,PRESION,0,[PRESION]
4,4,Vall-PC-FI003,anotaciones,1558-04-23,PRESION . ratura Se dio mandamiento de prisión...,ratura Se dio mandamiento de prisión a 23 de ...,0,"[ratura, Se, dio, mandamiento, de, prisión, a,..."
...,...,...,...,...,...,...,...,...
884,1026,Vall-PC-FC229,Fallo,1559-10-08,El Licenciado [PERSON]Francisco Vaca[PERSON] ....,(Rúbrica),0,[(Rúbrica)]
885,1027,Vall-PC-FC229,Fallo,1559-10-08,El Licenciado [PERSON]Francisco Vaca[PERSON] ....,(Rúbrica),0,[(Rúbrica)]
886,1028,Vall-PC-FC229,Fallo,1559-10-08,El Licenciado [PERSON]Francisco Vaca[PERSON] ....,(Rúbrica),0,[(Rúbrica)]
887,1030,Vall-PC-FC229,Fallo,1559-10-08,El Licenciado [PERSON]Francisco Vaca[PERSON] ....,Dada y pronunciada fue la dicha sentencia po...,10,"[Dada, y, pronunciada, fue, la, dicha, sentenc..."


## Preparation after statistical cleansing of the sample

In [20]:
df_1.to_excel('new_dataset_relations_diego_changes_13-02-2024.xlsx')

In [ ]:
import pandas as pd 
import regex as re 
df = pd.read_excel()